# LTI faculty

In [45]:
import pandas as pd
urls = [r'https://lti.cs.cmu.edu/directory/all/154/1?page=0', r'https://lti.cs.cmu.edu/directory/all/154/1?page=1']
datas = []
for url in urls:
    tables = pd.read_html(url) # Returns list of all tables on page
    datas.append(tables[0]) # Select table of interest


In [46]:
import numpy as np 
def parse_person_info(a):
    try:
        if np.isnan(a):
            return dict()
    except:
        pass
    
    data_dict = dict()
    names = ['Email', 'Office', 'Phone', 'Research Areas']
    idx = []
    for name in names: 
        try:
            idx.append(a.index(name))
        except:
            pass
    idx = list(sorted(idx))
    
    #parse first & second name 
    name_and_title = a[:idx[0]]
    name = ' '.join(name_and_title.split(' ')[:2])
    title = ' '.join(name_and_title.split(' ')[2:])
    data_dict['Name'] = name 
    data_dict['Title'] = title.strip()
    #parse other info
    for i in range(len(idx)):
        if i == len(idx) - 1:
            first, second = idx[i], len(a)
        else:
            first, second = idx[i], idx[i+1] 
        s = a[first:second]
        s = s.split(':')
        data_dict[s[0]] = s[1].strip()
    
    for name in names:
        if name not in data_dict:
            data_dict[name] = None
    return data_dict


In [105]:
#dict version 
lti_faculty = []
for data in datas:
    for d in data.values:
        for info in d:
            info_dict = parse_person_info(info)
            if info_dict:
                lti_faculty.append(info_dict)

In [47]:
lti_faculty = []
for data in datas:
    for d in data.values:
        for info in d:
            try:
                if np.isnan(info):
                    pass
            except:
                lti_faculty.append(info)

In [107]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/lti_faculty.json', 'w') as f:
    json.dump(lti_faculty, f)

# LTI Staff

In [1]:
import pandas as pd
urls = [r'https://lti.cs.cmu.edu/directory/all/154/2?page=0', r'https://lti.cs.cmu.edu/directory/all/154/2?page=1']
datas = []
for url in urls:
    tables = pd.read_html(url) # Returns list of all tables on page
    datas.append(tables[0]) # Select table of interest


/tmp/ipykernel_4101526/2916385793.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
lti_staff = []
for data in datas:
    for d in data.values:
        for info in d:
            try:
                if np.isnan(info):
                    pass
            except:
                lti_staff.append(info)

In [3]:
len(lti_staff)

34

In [4]:
lti_staff
['name', 'title', 'email', 'phone', 'office']

['Joan Axelson Office Manager Email: jaxelson@andrew.cmu.edu Phone: 412-268-7517',
 'Allison Blair Associate Business Manager Email: ablair2@cs.cmu.edu Office: 5715 TCS Hall Phone: 412-268-5489',
 'Bryan Burtner Associate Business Manager Email: bburtner@cs.cmu.edu Office: 6413 TCS Hall Phone: 412-268-2805',
 'Zhi-Qi Cheng Hauptmann Lab/Postdoctoral Research Associate Email: zhiqic@andrew.cmu.edu',
 'Jennifer Clark Administrative Coordinator Email: jclark2@andrew.cmu.edu',
 'Brianna Eriksen Academic Program Manager Email: bfreema2@andrew.cmu.edu Phone: 412-268-4277',
 'John Friday Sr. Administrative Coordinator Email: jfriday@andrew.cmu.edu Office: 5715 Gates & Hillman Centers Phone: 412-268-1810',
 'Daniel Gongaware Sponsored Research Administrator Email: dgongawa@cs.cmu.edu Office: 355 TCS Hall Phone: 412-268-6195',
 'Robin Hammer Levin Lab/Project Coordinator Email: robinham@andrew.cmu.edu',
 'Annette Han Rose Lab/Data Science Course Developer Email: ahan2@andrew.cmu.edu',
 'Susan E

In [112]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/lti_staff.json', 'w') as f:
    json.dump(lti_staff, f)

# LTI student (ignore now)

In [113]:
import pandas as pd
urls = [r'https://lti.cs.cmu.edu/directory/students/current/all']
datas = []
for url in urls:
    tables = pd.read_html(url) # Returns list of all tables on page
    datas.append(tables[0]) # Select table of interest


In [114]:
lti_student = []
for data in datas:
    for d in data.values:
        for info in d:
            try:
                if np.isnan(info):
                    pass
            except:
                lti_student.append(info)

# LTI Faculty paper

In [53]:
import requests

In [54]:
def is_cmu(faculty, threshold = 0.7):
    total = 0
    cs = 0
    for p in faculty['papers']:
        total += 1
        if p['fieldsOfStudy'] is not None and 'Computer Science' in p['fieldsOfStudy']:
            cs += 1
    if total == 0:
        return False
    return cs/total >= threshold


In [78]:
def extract_paper_info(faculty):
    papers = faculty['papers']
    paper_info = []
    for p in papers:
        if p['year'] is None or p['year'] < 2023:
            continue
        paper_string = ''
        paper_string += f'Author: {faculty["name"]} '
        paper_string += f'Title: {p["title"]} '
        paper_string += f'Publication year: {p["year"]} '
        coauthor = ''
        for a in p['authors']:
            coauthor += a['name'] + ', '
        coauthor = coauthor[:-2]
        paper_string += f'Coauthors: {coauthor} '
        if p['abstract'] is not None:
            paper_string += f'Abstract: {p["abstract"]}'
        paper_info.append(paper_string)
    return paper_info

In [79]:
faculty = json.load(open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/lti_faculty.json'))

In [80]:
faculty_names = [s.split(' ')[:2] for s in faculty]

In [82]:
publications = []
for faculty_name in faculty_names:
    if faculty_name[0] != "Eric":
        name_str = faculty_name[0]+ '+' +faculty_name[1]
        name_str = name_str.lower()
    else:
        name_str = 'eric+xing' #hardcode for eric ;)
    url = f'https://api.semanticscholar.org/graph/v1/author/search?query={name_str}&fields=name,aliases,url,affiliations,papers.abstract,papers.title,papers.authors,papers.year,papers.fieldsOfStudy'
    faculty_list = requests.get(url).json()
    try:
        faculty_list = faculty_list['data']
    except:
        continue
    for f in faculty_list:
        if is_cmu(f):
            publications.extend(extract_paper_info(f))

In [84]:
publications

['Author: Yonatan Bisk Title: HomeRobot: An Open Source Software Stack for Mobile Manipulation Research Publication year: 2024 Coauthors: Chris Paxton, Austin Wang, Binit Shah, Blaine Matulevich, Dhruv Shah, Karmesh Yadav, Santhosh Ramakrishnan, Sriram Yenamandra, Yonatan Bisk Abstract: Reproducibility in robotics research requires capable, shared hardware platforms which can be used for a wide variety of research. We’ve seen the power of these sorts of shared platforms in more general machine learning research, where there is constant iteration on shared AI platforms like PyTorch. To be able to make rapid progress in robotics in the same way, we propose that we need: (1) shared real-world platforms which allow different teams to test and compare methods at low cost; (2) challenging simulations that reflect real-world environments and especially can drive perception and planning research; and (3) low-cost platforms with enough software to get started addressing all of these problems. T

In [85]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/faculty_publication.json', 'w') as f:
    json.dump(publications, f)

# CMU Schedule of class

In [1]:

import pandas as pd
urls = [r'https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_spring.htm', r'https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_summer_1.htm', r'https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_summer_2.htm', r'https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_fall.htm']
datas = []
for url in urls:
    tables = pd.read_html(url) # Returns list of all tables on page
    datas.append(tables[0]) # Select table of interest
order = ['Spring', 'Summer 1/Summer All', 'Summer 2', 'Fall']

/tmp/ipykernel_1699649/1010705020.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
days_expand = {
    'M': 'Monday',
    'T': 'Tuesday',
    'W': 'Wednesday',
    'R': 'Thursday',
    'F': 'Friday',
    'S': 'Saturday',
    'U': 'Sunday'
}

In [12]:
courses = []
for table, semester in zip(datas, order):
    #drop first row and last column
    table = table.iloc[1:, :-1]
    table.columns = table.iloc[0]
    table = table.iloc[1:, :]

    #exclude rows where location and title is nan
    table = table[table['Location'].notna() | table['Title'].notna()]
    table.fillna(method='ffill', inplace=True)
    table.reset_index(inplace=True, drop=True)

    for i in range(len(table)):
        course_info = table.iloc[i].to_dict()
        s = semester + ' offering: '
        days_str = ''
        for k, v in course_info.items():
            #expand days symbol
            if k == 'Days':
                if v == 'TBA':
                    v = 'To be announced'
                else:
                    for day in v:
                        days_str += days_expand[day] + ', '
                    v = days_str[:-2]

            #expand lec/sec 
            if k == 'Lec/Sec':
                if v == 'Lec':
                    v = 'Lecture'
                else:
                    v = 'Section ' + v
            s += f'{k}: {v} '
        courses.append(s)
    


/tmp/ipykernel_1699649/116906015.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  table.fillna(method='ffill', inplace=True)
/tmp/ipykernel_1699649/116906015.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  table.fillna(method='ffill', inplace=True)
/tmp/ipykernel_1699649/116906015.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  table.fillna(method='ffill', inplace=True)
/tmp/ipykernel_1699649/116906015.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  table.fillna(method='ffill', inplace=True)


In [13]:
courses[0]

'Spring offering: Course: 48105 Title: Architecture Design Studio: Poiesis Studio 2 Units: 15.0 Lec/Sec: Lecture Days: Monday, Wednesday, Friday,  Begin: 02:00PM End: 04:50PM Bldg/Room: MM A14 Location: Pittsburgh, Pennsylvania Instructor(s): Yang '

In [77]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/course_schedule.json', 'w') as f:
    json.dump(courses, f)

# CMU Academic calendar

In [ ]:
import pandas as pd 
semesters = ['Fall 2023', 'Spring 2024', 'Summer One_All 2024', 'Summer Two 2024']
events = []
for semester in semesters:
    df = pd.read_excel(f'/zfsauton2/home/yifuc/11711-RAG/data/cmu/raw/{semester}.xlsx')
    for i in range(len(df)):
        date_dict = df.iloc[i].to_dict()
        #convert timestamp to string and remove hour, minute, and second
        date1 = date_dict['Date'].strftime('%Y-%m-%d')
        date2 = date_dict['Date2']
        #check for consecutive date
        if not str(date2) == 'nan' and not str(date2) == 'NaT':
            date2 = date2.strftime('%Y-%m-%d')
            date = f'{date1}-{date2}'
        else:
            date = date1
        #create date string 
        date_string = f'Date: {date}'
        for k, v in date_dict.items():
            if k not in ['Date', 'Date2', '-']:
                date_string += f' {k}: {v}'
        date_string = semester + ': ' + date_string
        events.append(date_string)

In [21]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/university_calendar.json', 'w') as f:
    json.dump(events, f)

# LTI Academic

In [1]:
import pandas as pd
urls = [r'https://lti.cs.cmu.edu/learn']
datas = []
for url in urls:
    tables = pd.read_html(url) # Returns list of all tables on page
    datas.append(tables[0]) # Select table of interest


/tmp/ipykernel_521683/3081452667.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
url = r'https://lti.cs.cmu.edu/learn'
#parse the first chunk of the html about phd program in lti 
import requests
from bs4 import BeautifulSoup
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
phd = soup.find_all('div', class_='field-item even')

In [12]:
phd[3].text

'Carnegie Mellon\'s School of Computer Science has a centralized online application process. Applications and all supporting documentation for fall admission to any of the LTI\'s graduate programs must be received by the application deadline. Incomplete applications will not be considered. The application period for Fall 2024 will open on September 6, 2023.\nFinal Application Deadline\nDecember 13, 2023 at 3 p.m. EST.\nCost\n$100 per program and $80 if the applicant applies before November 29, 2023 at 3 p.m. EST (early deadline).\nFee Waivers\nFee waivers may be available in cases of financial hardship. For more information, please refer to the\xa0School of Computer Science Fee Waiver page.\nRequirements\nThe School of Computer Science requires the following for all Ph.D. applications.\nGRE scores:\xa0GREs are now optional, but if you want to submit GRE scores:These must be less than five years old. The GRE Subject Test is not required, but is recommended. Our Institution Code is 2074;

In [21]:
texts = [
''' 
Ph.D. in Language and Information Technology: The Ph.D. in LTI focuses on developing the next generation of scientific and entrepreneurial leaders. The first two years of the Ph.D. program are similar to our MLT program. After the second year, you will spend most of your time working closely with your faculty advisor on research that advances the state-of-the-art in computer science.

Ph.D. students are expected to publish papers about original research in the most competitive scientific journals and international conference proceedings, and to present their research at conferences and workshops. Most of our Ph.D. graduates become professors and research scientists, while a few have started their own companies.
''',
''' 
Ph.D. in Language and Information Technology Requirements: In general, students pursuing a Ph.D. in Language and Information Technologies must

    Pass at least 96 units of graduate-level courses.
    Satisfy proficiencies in writing, presentation, programming and teaching; and
    Propose, write and defend a Ph.D. dissertation (thesis).

Students must also attend the LTI Colloquium each semester and satisfy our Research Speaking Requirement.

For a detailed breakdown of the above requirements, download and read the PhD Handbook.
''',
''' 
Ph.D. in Language and Information Technology Curriculum: In order to obtain your Ph.D. in Language and Information Technologies, you need to pass 96 units (generally, eight courses) of graduate courses that fulfill these requirements:

    At least 72 units of LTI courses: Must include one class in each LTI focus area.
    At least 24 units of SCS courses.
    At least two lab courses in two different research areas.
''',
''' 
Ph.D. in Language and Information Technology Admission: 

Carnegie Mellon's School of Computer Science has a centralized online application process. Applications and all supporting documentation for fall admission to any of the LTI's graduate programs must be received by the application deadline. Incomplete applications will not be considered. The application period for Fall 2024 will open on September 6, 2023.

Final Application Deadline

December 13, 2023 at 3 p.m. EST.

Cost

    $100 per program and $80 if the applicant applies before November 29, 2023 at 3 p.m. EST (early deadline).

Fee Waivers

Fee waivers may be available in cases of financial hardship. For more information, please refer to the School of Computer Science Fee Waiver page.

Requirements

The School of Computer Science requires the following for all Ph.D. applications.

    GRE scores: GREs are now optional, but if you want to submit GRE scores:
    These must be less than five years old. The GRE Subject Test is not required, but is recommended. Our Institution Code is 2074; Department Code is 0402.
    English Proficiency Requirement: If you will be studying on an F-1 or J-1 visa, and English is not a native language for you (native language…meaning spoken at home and from birth), an official copy of an English proficiency score report is required. The English proficiency requirement cannot be waived for any reason. Find more information under "Test Scores" on our FAQ page.
    Successful applicants will have a minimum TOEFL score of 100. Our Institution Code is 4256; the Department Code is 78.
    Official transcripts from each university you have attended, regardless of whether you received your degree there.
    Current resume.
    Statement of Purpose.
    Three letters of recommendation.

For more details on these requirements, please see the SCS Doctoral Admissions page.

In addition to the SCS guidelines, the LTI requires:

    A short (1-3 minute) video of yourself. There will be a prompt question that you will respond to. You will have three attempts. This is not a required part of the application process, but it's strongly suggested.
    Any outside funding you are receiving must be accompanied by an official award letter.

No incomplete applications will be eligible for consideration.

For specific application/admissions questions, please contact Stacey Young.
''',
''' 
Dual-Degree Ph.D. in Language and Information Technologies (Portugal Partnership): The LTI offers a dual-degree Ph.D. in Language and Information Technologies in cooperation with:

    Universidade de Aveiro (Ph.D. in Computer Engineering), Universidade do Minho (Ph.D. in Informatics)  and the Universidade do Porto (FCUP, Ph.D. in Computer Science and FEUP, Ph.D. in Computer Science) as part of MAPi;
    Universidade de Lisboa, Faculdade de Ciências – FCUL (Ph.D. in Informatics) 

    Universidade de Lisboa, Instituto Superior Técnico – IST  (Ph.D. in Computer Science and Engineering, Ph.D. in Electrical and Computer Engineering, Ph.D. in Information Security)

    Universidade Nova de Lisboa, Faculdade de Ciências e Tecnologia – FCTUNL (Ph.D. in Computer Science)

    Universidade de Coimbra, Faculdade de Ciências e Tecnologia – FCTUC (Ph.D. in Information Science and Technology)

Students jointly enrolled in the LTI Ph.D program spend a year in Portugal, then two years at Carnegie Mellon taking classes in linguistics, computer science, statistical learning and task orientation.

After completing the majority of their academic requirements, students return to Portugal for the next two years to conduct extensive research, ultimately leading to a dissertation topic that will be publicly defended. One adviser from each institution co-supervises their student’s progress and helps to define their final thesis topic.
''',
''' 
Dual-Degree Ph.D. in Language and Information Technologies (Portugal Partnership) Requirments: Students participating in the dual-degree program will spend their first year in Portugal, followed by two years in Pittsburgh to complete their coursework. They will complete a maximum of eight courses with a proper balance of focus areas (linguistics, computer science, statistical/learning and task orientation). After that, they will return to Portugal for their last two years, pursuing research and completing their dissertation. For more, see the Carnegie Mellon | Portugal page.
''',
''' 
Dual-Degree Ph.D. in Language and Information Technologies (Portugal Partnership) Admission: 

Students applying to the dual degree program must apply through Carnegie Mellon's online application. In addition to the requirements listed below, prospective students must also contact Stacey Young when applying.

Carnegie Mellon's School of Computer Science has a centralized online application process. Applications and all supporting documentation for fall admission to any of the LTI's graduate programs must be received by the application deadline. Incomplete applications will not be considered. The application period for Fall 2024 will open on September 6, 2023.

Final Application Deadline

December 13, 2023 3:00 p.m. EST.

Cost

    $100 per program, $80 for applications submitted by November 29, 2023 at 3:00 p.m. EST (early deadline).

Fee Waivers

Fee waivers may be available in cases of financial hardship, or for participants in select programs. For more information, please refer to the School of Computer Science Fee Waiver page.

Requirements

The School of Computer Science requires the following for all Ph.D. applications.

    GRE scores: These must be less than five years old. The GRE Subject Test is not required, but is recommended. Our Institution Code is 2074; Department Code is 0402.
    English Proficiency Requirement: If you will be studying on an F-1 or J-1 visa, and English is not a native language for you (native language…meaning spoken at home and from birth), an official copy of an English proficiency score report is required. The English proficiency requirement cannot be waived for any reason. Find more information under "Test Scores" on our FAQ page.
    Successful applicants will have a minimum TOEFL score of 100. Our Institution Code is 4256; the Department Code is 78.
    Official transcripts from each university you have attended, regardless of whether you received your degree there.
    Current resume.
    Statement of Purpose.
    Three letters of recommendation.

For more details on these requirements, please see the SCS Doctoral Admissions page.

In addition to the SCS guidelines, the LTI requires:

    A short (1-3 minute) video of yourself. There will be a prompt question that you will respond to. You will have three attempts. This is not a required part of the application process, but it's strongly suggested.

    Any outside funding you are receiving must be accompanied by an official award letter.

No incomplete applications will be eligible for consideration.

For specific application/admissions questions, please contact Stacey Young.

''',
''' 
Master of Language Technologies: The MLT program prepares students for a research career in academia or industry. In this program, you’ll be immersed in research for two full years. During the academic year, your time will be evenly split between taking courses and doing research with your faculty advisor. Your summer will be devoted entirely to research. Many MLT grads continue on to Ph.D. programs at CMU and other top institutions, while others pursue careers at companies emphasizing research and rapid innovation.
''',
''' 
Master of Language Technologies Requirements: 

The MLT program lasts two years (24 months), and students must complete two summers of research. Students should usually expect to graduate in August of their second year.

MLT students take 120 or more course units (about 10 courses), at least 72 of which are LTI courses, and 24 of which are School of Computer Science (SCS) courses. Most of these are 12-unit courses, although lab courses are typically 6 units. Our courses generally assume knowledge of programming and data structures. The remaining units may also be taken from the LTI, or with approval from the faculty advisor, any other senior- or graduate-level course offered at CMU or Pitt.

Directed research is another integral part of the MLT program; MLT students carry out directed research during their studies, with guidance from their faculty advisors.

Students may also choose to complete an optional MLT thesis. Guidelines can be found in the PDF iconMLT Handbook.

''',
''' 
Master of Language Technologies Admission: 

Carnegie Mellon's School of Computer Science has a centralized online application process. Applications and all supporting documentation for fall admission to any of the LTI's graduate programs must be received by the application deadline. Incomplete applications will not be considered. The application period for Fall 2024 will open on September 6, 2023.

*Please note, we no longer require mailed, hard versions of transcripts or test scores at the time of application. Do not mail anything to the admissions office. If you are accepted to a program, you will be given instruction to then mail your materials.

Final Application Deadline

December 13, 2023 at 3 p.m. EST.

Cost

    $100 per program, $80 for applications submitted before November 29, 2023 at 3PM EST (early deadline).

Fee Waivers

Fee waivers may be available in cases of financial hardship, or for participants in select "pipeline" programs. For more information, please refer to the School of Computer Science Fee Waiver page.

Requirements

The School of Computer Science requires the following for all graduate program applications:

    GRE scores: GREs are now optional, but if you want to submit GRE scores:
    These must be less than five years old. A GRE subject test in science, engineering, computer science, math, etc. is not required, but you may complete one and submit the scores if you wish. Our Institution Code is 2074; Department Code is 0402.
    If you will be studying on an F-1 or J-1 visa, and English is not a native language for you (native language…meaning spoken at home and from birth), an official copy of an English proficiency score report is required. The English proficiency requirement cannot be waived for any reason. Find more information under "Test Scores" on our FAQ page.
    Successful applicants will have a minimum TOEFL score of 100. Our Institution Code is 4256; the Department Code is 78.
    Official transcripts from each university you have attended, regardless of whether you received your degree there.
    Current resume.
    Statement of Purpose.
    Three letters of recommendation.

For more details on these requirements, please see the SCS Master's Admissions page.

In addition to the SCS guidelines, the LTI requires:

    A short (1-3 minute) video of yourself. There will be a prompt question that you will respond to. You will have three attempts. This is not a required part of the application process, but it's strongly suggested.

    Any outside funding you are receiving must be accompanied by an official award letter.

No incomplete applications will be eligible for consideration.

For specific application/admissions questions, please contact Kate Schaich.

Tuition Rates

Tuition is set by the School of Computer Science and can vary by year. Current tuition rates can be found on the Graduate Tuition section of the Student Financial Services website.

Financial Aid Resources

Research Assistant-ships are occasionally offered by research advisors to current MLT students. These are not guaranteed and vary from semester-to-semester. They fluctuate and are dependent on the funding source, research advisor and MLT student.

Student Financial Services has additional information on financial aid and billing / payments. They have a detailed outline of how to apply for financial aid on the Graduate Financial Aid Process page.

Enrollment & Finances has additional resource links to assist with financial aid and tuition payments.

Graduate Education – Financial Assistance provides resources for current students regarding emergency loans and conference travel grants.

''',
''' 
Master of Science in Intelligent Information Systems (MIIS): The Master's in Intelligent Information Systems degree focuses on recognizing and extracting meaning from text, spoken language and video. As an MIIS student, you’ll receive the department’s deepest exposure to content analysis and machine learning. In addition to completing the program’s coursework, you’ll work on directed study projects with your faculty advisor for two semesters; participate in a summer internship; and collaborate with your peers on a semester-long, group-oriented capstone project. This combination of classroom instruction, professional experience, and using new skills in significant projects with world-class colleagues will help prepare you for a successful career in industry or government. Our alumni have gone on to exciting careers at places like Apple, IBM and Google, and most have job offers within six weeks of graduation.
''',
''' 
Master of Science in Intelligent Information Systems (MIIS) Requirements: 

The Intelligent Information Systems degree offers students the flexibility to create their own course of study in consultation with their advisor. 

MIIS students gain three types of practical experience: software development supervised by their advisor (24 units equivalent to two courses); a summer internship (which can be waived for students that have sufficient prior professional experience); and a capstone project executed in a group of peers (42 units equivalent to three 12-unit courses and one 6-unit course). This combination is proven to help IIS students to broaden their skills quickly. The MIIS degree is offered in two options:

Option 1. Standard MIIS degree (MIIS-16) - A 16-month track that is completed in three academic semesters (fall, spring, fall) and a summer internship.  

Option 2. MIIS: Advanced Study degree (MIIS-21) - A 21-month track that is completed in four academic semesters (fall, spring, fall, spring) and a summer internship.

MIIS: Advanced Study track offers indepth degree in one of the following areas of concentration:

    Human Language for Language Technologies
    Language Technology Application
    Machine Learning for Language Technologies

Part-time education option is available in some cases. 

MIIS-16 students must take at least 84 units (typically 7 courses) of qualifying and elective courses that satisfy human language, machine learning, and language technology applications breadth requirements. MIIS-21 students have to take at least two more courses from the selected concentration area to satisfy their degree requirements, making it total of 108 units (typically 9 courses) of qualifying and elective courses, that also satisfy breadth requirements.

For a full list of requirements, read the MIIS Handbook.

''',
''' 
Master of Science in Intelligent Information Systems (MIIS) Admission: 

Carnegie Mellon's School of Computer Science has a centralized online application process. Applications and all supporting documentation for fall admission to any of the LTI's graduate programs must be received by the application deadline. Incomplete applications will not be considered. The application period for Fall 2024 will open on September 6, 2023.

*Please note, we no longer require mailed, hard versions of transcripts or test scores at the time of application. Do not mail anything to the admissions office. If you are accepted to a program, you will be given instruction to then mail your materials.

Final Application Deadline

December 13, 2023 at 3:00 p.m. EST.

Cost

    $100 per program, $80 for applications submitted before November 29, 2023 at 3:00 p.m. EST (early deadline).

Fee Waivers

Fee waivers may be available in cases of financial hardship, or for participants in select "pipeline" programs. For more information, please refer to the School of Computer Science Fee Waiver page.

Requirements

The School of Computer Science requires the following for all Master's applications.

    GRE scores: MIIS applicants must submit their GRE scores. The scores must be less than five years old. The GRE Subject Test is not required, but is recommended. Our Institution Code is 2074; Department Code is 0402.

    Proof of English Language Proficiency:

If you will be studying on an F-1 or J-1 visa, and English is not a native language for you (native language…meaning spoken at home and from birth), we are required to formally evaluate your English proficiency. We require applicants who will be studying on an F-1 or J-1 visa, and for whom English is not a native language, to demonstrate English proficiency via one of these standardized tests: TOEFL (preferred), IELTS, or Duolingo.  We discourage the use of the "TOEFL ITP Plus for China," since speaking is not scored.

We do not issue waivers for non-native speakers of English.  In particular, we do not issue waivers based on previous study at a U.S. high school, college, or university.  We also do not issue waivers based on previous study at an English-language high school, college, or university outside of the United States.  No amount of educational experience in English, regardless of which country it occurred in, will result in a test waiver. Submit valid, recent scores: If as described above you are required to submit proof of English proficiency, your TOEFL, IELTS or Duolingo test scores will be considered valid as follows: If you have not received a bachelor’s degree in the U.S., you will need to submit an English proficiency score no older than two years. (scores from exams taken before Sept. 1, 2021, will not be accepted.)

If you are currently working on or have received a bachelor's and/or a master's degree in the U.S., you may submit an expired test score up to five years old. (scores from exams taken before Sept. 1, 2018, will not be accepted.)Additional details about English proficiency requirements are provided on the FAQ page.

    Official transcripts from each university you have attended, regardless of whether you received your degree there.
    Current resume.
    Statement of Purpose.
    Three letters of recommendation.

For more details on these requirements, please see the SCS Master's Admissions page.

In addition to the SCS guidelines, the MIIS requires:

    A short (1-3 minute) video of yourself. There will be a prompt question that you will respond to. You will have three attempts.

No incomplete applications will be eligible for consideration.

For specific application/admissions questions, please contact Brianna Eriksen.

''', 
''' 
Master of Computational Data Science (MCDS): The MCDS degree focuses on engineering and deploying large-scale information systems. Our comprehensive curriculum equips you with the skills and knowledge to develop the layers of technology involved in the next generation of massive information system deployments and analyze the data these systems generate. When you graduate, you’ll have a unified vision of these systems from your core courses; internship experience; and semester-long, group-oriented capstone project. MCDS graduates are sought-after software engineers, data scientists and project managers at leading information technology, software services and social media companies.
''',
''' 

Master of Computational Data Science (MCDS) Requirements:

The MCDS program offers three majors: Systems, Analytics, and Human-Centered Data Science. All three require the same total number of course credits, split among required core courses, electives, data science seminar and capstone courses specifically defined for each major. The degree can also be earned in two different ways, depending on the length of time you spend working on it. Regardless of the timing option, all MCDS students must complete a minimum of 144 units to graduate.

Here are the options:

    Standard Timing — a 16-month degree consisting of study for fall and spring semesters, a summer internship, and fall semester of study. Each semester comprises a minimum of 48 units. This timing is typical for most students. Students graduate in December.
    Extended Timing — a 20-month degree consisting of study for fall and spring semesters, a summer internship, and a second year of fall and spring study. Each semester comprises a minimum of 36 units. Students graduate in May.

For a complete overview of the MCDS requirements, visit the MCDS website or read the MCDS Handbook.

''',
''' 
Master of Computational Data Science (MCDS) Admission:


Carnegie Mellon's School of Computer Science has a centralized online application process. Applications and all supporting documentation for fall admission to any of the LTI's graduate programs must be received by the application deadline. Incomplete applications will not be considered. The application period for Fall 2024 will open on September 6, 2023.

*Please note, we no longer require mailed, hard versions of transcripts or test scores at the time of application. Do not mail anything to the admissions office. If you are accepted to a program, you will be given instruction to then mail your materials.

Final Application Deadline

December 13, 2023 at 3:00 p.m. EST.

Cost

    $100 per program, $80 for applications submitted before November 29,2023 at 3:00 p.m. EST (early deadline).

Fee Waivers

Fee waivers may be available in cases of financial hardship, or for participants in select "pipeline" programs. For more information, please refer to the School of Computer Science Fee Waiver page.

Requirements

The School of Computer Science requires the following for all Ph.D. applications.

    GRE scores: These must be less than five years old. The GRE Subject Test is not required, but is recommended. Our Institution Code is 2074; Department Code is 0402.

    Proof of English Language Proficiency:
    If you will be studying on an F-1 or J-1 visa, and English is not a native language for you (native language…meaning spoken at home and from birth), we are required to formally evaluate your English proficiency. We require applicants who will be studying on an F-1 or J-1 visa, and for whom English is not a native language, to demonstrate English proficiency via one of these standardized tests: TOEFL (preferred), IELTS, or Duolingo. We discourage the use of the "TOEFL ITP Plus for China," since speaking is not scored.

    We do not issue waivers for non-native speakers of English. In particular, we do not issue waivers based on previous study at a U.S. high school, college, or university. We also do not issue waivers based on previous study at an English-language high school, college, or university outside of the United States. No amount of educational experience in English, regardless of which country it occurred in, will result in a test waiver.

    Applicants applying to MCDS are required to submit scores from an English proficiency exam taken within the last two years. Scores taken before Sept. 1, 2021, will not be accepted regardless of whether you have previously studied in the U.S. For more information about their English proficiency score policies, visit the MCDS admission website. 

    Successful applicants will have a minimum TOEFL score of 100 (Reading, Listening, Speaking, Writing scores all 25 or above), IELTS score of 7.5 (Reading 7 or above, Listening 7 or above, Speaking 7.5 or above, Writing 6.5 or above), or DuoLingo score of 120 (Literacy 115 or above, Comprehension 125 or above, Production 100 or above, Conversation 105 or above). Our Institution Code is 4256; the Department Code is 78. Additional details about English proficiency requirements are provided on the FAQ page. 
    Official transcripts from each university you have attended, regardless of whether you received your degree there.
    A short (1-3 minutes) video of yourself. Tell us about you and why you are interested in the MCDS program. This is not a required part of the application process, but it is STRONGLY suggested.  
    Current resume.
    Statement of Purpose.
    Three letters of recommendation.

For more details on these requirements, please see the SCS Master's Admissions page.

In addition to the SCS guidelines, the LTI requires:

    Any outside funding you are receiving must be accompanied by an official award letter.

No incomplete applications will be eligible for consideration.

For specific application/admissions questions, please contact Jennifer Lucas or Caitlin Korpus.

''',
''' 
Master of Computational Data Science (MCDS) Curriculum:

To earn an MCDS degree, student must pass courses in the core curriculum, the MCDS seminar, a concentration area and electives. Students must also complete a capstone project in which they work on a research project at CMU or on an industry-sponsored project.

In total, students must complete 144 eligible units of study, including eight 12-unit courses, two 12-unit seminar courses and one 24-unit capstone course. Students must choose at minimum five core courses. The remainder of the 12-unit courses with course numbers 600 or greater can be electives chosen from the SCS course catalog. Any additional non-prerequisite units taken beyond the 144 units are also considered electives.

MCDS students must also pass the undergraduate course 15-513 Introduction to Computer Systems (6 units), typically in the summer before their program commences. The student must pass with a grade of B- or better. Failure to pass the course means that the student takes 15-213 during either the fall or spring semester. Note that in both cases the units do not count toward the 144 eligible units of study.
''',
''' 
Master of Science in Artificial Intelligence and Innovation (MSAII):
The Master of Science in Artificial Intelligence and Innovation (MSAII) program is a successor to the M.S. in Biotechnology, Innovation and Computing (MSBIC). It combines a rigorous AI and machine learning curriculum with real-world team experience in identifying an AI market niche and developing a responsive product in cooperation with external stakeholders. The core program, which lasts four semesters and leads to a capstone project, focuses on both intrapreneurship and entrepreneurship, equipping graduates to either begin a startup or develop a new organization within an existing company. Students will also gain critical practical skills, such as making persuasive technical presentations, assembling development teams, and evaluating the potential of new market ideas.
''',
''' 
Master of Science in Artificial Intelligence and Innovation (MSAII) Requirements:


Incoming students generally hold undergraduate degrees in computer science, software engineering, bioinformatics or bioengineering. To earn the MSAII degree, you must pass courses in the Core Curriculum, the Knowledge Requirements and Electives. You must also complete a capstone project in which you work on a development project as part of the Core Curriculum. In total, you will complete 192 eligible units of study, including 84 units of Core Curriculum (including the 36-unit Capstone), 72 units of Knowledge Requirements and at least 36 units of approved Electives.

For full requirements and program details, read the MSAII Handbook.

''',
''' 

Master of Science in Artificial Intelligence and Innovation (MSAII) Admission: 

Carnegie Mellon's School of Computer Science has a centralized online application process. Applications and all supporting documentation for fall admission to any of the LTI's graduate programs must be received by the application deadline. Incomplete applications will not be considered. The application period for Fall 2024 will open on September 6, 2023.

Final Application Deadline

December 13, 2023 at 3:00 p.m. EST.

Cost

    $100 for one program, $80 if application is submitted before November 29, 2023 at 3PM EST (early deadline).

Fee Waivers

Fee waivers may be available in cases of financial hardship, or for participants in select "pipeline" programs. For more information, please refer to the School of Computer Science Fee Waiver page.

Requirements

The School of Computer Science requires the following for all applications:

    A GPA of 3.0 or higher. (Students should report raw university GPA scores and NOT converted scores. Please DO NOT convert your international score to a US GPA or weighted GPA or other system).

    GRE scores: GRE is required. Our Institution Code is 2074; Department Code is 0402. 

    English Language Proficiency: If you will be studying on an F-1 or J-1 visa, and English is not a native language for you (native language…meaning spoken at home and from birth), an official copy of an English proficiency score report is required. The English proficiency requirement cannot be waived for any reason. Find more information under "Test Scores" on our FAQ page.Unofficial transcripts from each university you have attended, regardless of whether you received a degree.

    Current resume.

    Statement of Purpose. A Statement of Purpose is not a resume. It should discuss your reasons for choosing the MSAII program and indicate your intended career path. 

    Three letters of recommendation.

    A short (1-3 minutes) video of yourself. Tell us about you and why you are interested in the MSAII program. This is not a required part of the application process, but it is STRONGLY suggested.  

For specific application/admissions questions, please contact Amber Vivis. 
''',

''' 
Undergraduate LT Concentration: Human language technologies have become an increasingly central component of computer science. Information retrieval, machine translation and speech technology are used daily by the general public, while text mining, natural language processing and language-based tutoring are common within more specialized professional or educational environments. The LTI prepares students for this world by offering a minor that gives you the opportunity to not only learn about language technologies, but to also apply that knowledge through a directed project.
''',

''' 
Undergraduate LT Concentration Requirements: 


Students interested in the language technologies minor must complete our prerequisite courses with an average grade of B (3.0) or better before applying to the program. (Students who do not meet this average must submit a letter of explanation along with their application.) Prerequisites include:

    Principles of Imperative Computation (15-122)
    Principles of Functional Programming (15-150)

We also strongly encourage candidates to take

    Differential and Integral Calculus (21-120) and Integration and Approximation (21-122)
    Matrices and Linear Transformations (21-241) or Matrix Theory (21-242)
    Probability and Computing (15-259) or Probability (21-325) or Probability Theory for Computer Scientists (36-218) or Introduction to Probability Theory (36-225)


''',
'''
Undergraduate LT Concentration Admission:
Students interested in earning a minor in language technologies must apply for admission no later than September 30 of their senior year. An admission decision will usually be made within one month. Students may petition the LTI undergraduate program director to be admitted to the minor earlier or later in their undergraduate careers. To apply, contact the program's director, Carolyn Rosé.
''',
''' 
Undergraduate LT Concentration Curriculum:
The Language Technologies Concentration requires that SCS students complete one core course and their choice of three elective courses of at least 9 units each. The electives can be chosen from a specific set of stand-alone courses. In addition to the four courses, students are required to do an undergraduate research project for at least 9 units to complete their concentration.
'''
]

In [22]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/LTI_Programs.json', 'w') as f:
    json.dump(texts, f)

# SCS 25 great things and history

In [1]:
import requests
from bs4 import BeautifulSoup

In [4]:
url = r'https://www.cs.cmu.edu/scs25/25things'
#parse the whole webpage's text
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
text = soup.get_text()
text = text.strip().split('\n')[-25:]

In [6]:
text = [t.strip() for t in text]
text

["What’s so great about computer science at Carnegie Mellon?We're glad you asked! Here are 25 great ideas from CMU computer scientists to think about as we celebrate the birthday of the School of Computer Science.1. Artificial intelligence, 1955-56\xa0Can you write a working computer program without a computer? Herb Simon (H’90), at left, Allen Newell (IA’57), at right, and Cliff Shaw did. The team created the first artificial intelligence program, Logic Theorist, which could solve logic puzzles in the same way that a human might solve them. Newell demonstrated that it worked by writing the instructions on 3-by-5 index cards that were manipulated on the kitchen table by Newell, his wife, and a group of Carnegie Tech grad students.",
 '2. Multi-core processors, 1971\xa0Multi-core processors are common in today’s computers, but they were still science fiction in the early 1970s. But when CMU researchers found their existing machines too slow to keep pace with the advance of speech and gr

In [7]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/scs_25_great_thing.json', 'w') as f:
    json.dump(text, f)

In [30]:
url = r'https://www.cs.cmu.edu/scs25/history'
#parse the whole webpage's text
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
text = soup.get_text()
text = text.strip().split('\n')[0] + '\n' + text.strip().split('\n')[-1]

A history of SCS | SCS25 - Carnegie Mellon University School of Computer Science
For an expanded history of the School of Computer Science and its predecessors at CMU, read "Institutional Memories" in the Summer 2014 issue of The Link magazine.In 2014, the School of Computer Science celebrated its 25th year as a stand-alone college within Carnegie Mellon University. It was arguably the first college devoted solely to computer science in the United States, and a model for others that followed. But CMU’s computer science era begins much earlier—in 1956, with the arrival of an IBM 650 computer on the campus of what was then known as Carnegie Institute of Technology. The IBM 650 had magnetic-drum memory and a processing speed of approximately 60 instructions per second. Herb Simon (H’90), associate dean of the Graduate School of Industrial Administration—now known as CMU’s Tepper School of Business—established Carnegie Tech’s first Computation Center with the help of its first director, Al

In [8]:
text = [
'''
A history of SCS

For an expanded history of the School of Computer Science and its predecessors at CMU, read "Institutional Memories" in the Summer 2014 issue of The Link magazine.

In 2014, the School of Computer Science celebrated its 25th year as a stand-alone college within Carnegie Mellon University. It was arguably the first college devoted solely to computer science in the United States, and a model for others that followed. But CMU’s computer science era begins much earlier—in 1956, with the arrival of an IBM 650 computer on the campus of what was then known as Carnegie Institute of Technology. The IBM 650 had magnetic-drum memory and a processing speed of approximately 60 instructions per second. Herb Simon (H’90), associate dean of the Graduate School of Industrial Administration—now known as CMU’s Tepper School of Business—established Carnegie Tech’s first Computation Center with the help of its first director, Alan Perlis (S’42).
''',
'''
First freshman-level computer science course

In 1956 and 1957, Simon, Allen Newell (IA'57) and Cliff Shaw of RAND designed the Logic Theorist, a computer program that could develop proofs for theorems in much the same way a human would work. They also developed linked-list data structures, the foundation of computer programming. Perlis, Simon and Newell are credited with defining the term “computer science” as “the theory and design of computers,” as well as (in Newell’s words) “the study of the phenomena arising from them.” In 1958, Perlis began teaching the first freshman-level computer programming course in the United States at Carnegie Tech.
''',
''' 
Computer science Ph.D. program created

In 1961, the Computation Center and its newest computer, a Bendix G-20, were moved to recently completed Scaife Hall. That same year, Carnegie Tech created an interdisciplinary Ph.D. program called Systems and Communications Sciences, combining elements of computer science, mathematics, psychology, business and electrical engineering. The university’s first computer science Ph.D.s were graduates of this program.
''',
''' 
Computer Science Department established

In 1965, Carnegie Tech established its Computer Science Department, or CSD, with a $5 million grant from the R.K. Mellon Foundation. Perlis was the first department head. There were no undergraduates; only Ph.D. students were admitted, and the department’s focus was on research, much of it funded by the federal government through the Defense Department’s Advanced Research Projects Agency, or DARPA. In 1967, Carnegie Institute of Technology merged with the nearby Mellon Institute of Industrial Research to form Carnegie-Mellon University. The Department of Computer Science moved into the newly created Mellon Institute of Science, later renamed Mellon College of Science, or MCS. Future SCS dean Raj Reddy joined the CSD in 1969 after three years as an assistant professor at Stanford. He brought with him research in speech, language and computer vision. But in 1970 and 1971, the new Computer Science Department faced its first crisis, as half of its tenured faculty members—including department head Perlis—left for other universities. Joe Traub was recruited from Bell Labs to CMU to become the new department head.
''',
''' 
Multi-processor machines emerge

CSD emerged from the brief crisis as a highly agile, interdisciplinary entity, with many new faculty members taking joint appointments with other CMU departments. Several large projects emerged in the Computer Science Department, including C.mmp, the first shared-memory multiprocessor computer, with 16 processing units, and Cm*, a 50-processor computer. These computers were the forerunners of today’s ubiquitous multi-core desktops and laptops.
''',
''' 
Turing Awards and a Nobel Prize

In 1975, Simon and Newell were awarded the A.M. Turing Award for their work in artificial intelligence. (As of 2014, 12 CMU alumni or faculty have been awarded Turings, sometimes considered the Nobel Prize of computing.) Three years later, Simon received the Nobel Prize in Economics for his work on decision-making theory. As the 1970s progressed, Newell became interested in human-computer interaction, and began a long relationship with Xerox’s Palo Alto Research Center, or PARC, which released the Xerox Alto in 1973. Considered a forerunner to many of the computing environments that followed, Alto featured a graphical-user interface and was among the first commercially available workstations controlled with a mouse. Inspired by Alto, Reddy launched a drive for development of CMU’s own “three-M” machine—a personal workstation with a megabyte of memory, a megapixel display and at least one million instructions per second of processing power.
''',
''' 
Launching a Robotics Institute

In 1979, an executive at Pittsburgh’s Westinghouse Electric Corp., Tom Murrin, collaborated with Jordan and Reddy to create the Robotics Institute, with Reddy as its first director. By 1982, the Computer Science Department included more than 30 faculty members and 100 graduate students.
''',
''' 
The best-wired campus in the world

Working with IBM in the early 1980s, the university and the Computer Science Department established another new research frontier: Development of a high-speed computer network that would reach virtually every room on campus, along with a GUI-based computing environment, and providing networked PCs or workstations for 7,000 students, faculty members and employees. Called the Andrew Project, it turned Carnegie Mellon into the best-connected, most-wired university in the world—a process Newell called “greening up the campus with computer science.” CMU also became home to a new Software Engineering Institute, funded by the Defense Department, to study computer security and develop best practices in the design of operating systems. Between 1982 and 1985, the amount of sponsored research in the Computer Science Department doubled, from $7.2 million to $15.3 million—more than the other four departments in the Mellon College of Science combined
''',
''' 
A “school of computer science” is proposed

Feeling that CSD’s needs were inadequately represented in MCS, CSD head A. Nico Habermann and then-CMU provost Angel Jordan in 1986 wrote a white paper proposing the creation of “a School of Computer Science.” Responding to concerns from the faculty that the change might be taking place too quickly, the university first established a free-floating Department of Computer Science. The experiment, which lasted two years, was an unqualified success. Separately, and also in 1986, the Pittsburgh Supercomputing Center was created as a joint effort between CMU, the University of Pittsburgh and Westinghouse Electric Corp.
''',
''' 
SCS is officially formed

CMU’s Faculty Senate in the fall of 1988 agreed to President Richard Cyert’s plan to elevate the Department of Computer Science to college status. In addition to the Computer Science Department, SCS also incorporated the Robotics Institute, the Center for Machine Translation, and researchers from the Information Technology Center, which had developed Andrew. On Dec. 13, 1988, Cyert (H’89) told faculty and staff that Habermann had been appointed CMU’s first Dean of Computer Science, effective Dec. 1, and that the School of Computer Science would soon begin operations. SCS made its formal debut on Dec. 22, 1988, with a reception in the Wherrett Room of Skibo Hall, CMU’s student union. The official announcement of CMU’s new “graduate School of Computer Science” was made Jan. 3, 1989.
''',
''' 
Undergraduate degrees begin

For several years, undergraduates interested in computer science pursued an “applied math/CS” bachelor’s degree offered by the Mathematics Department. CSD professor Mary Shaw (CS’72) led CMU’s first effort to design an undergraduate curriculum solely in computer science. She and her colleagues were guided by the Carnegie Plan—guidelines established in 1938 under Carnegie Tech President Robert Doherty (A’40, E’48, H’50), outlining the principles of a sound professional education. Drawing on Shaw’s plan and also on the work of other faculty members, an undergraduate program in computer science was created during the 1989-90 academic year. Seven CS majors were admitted to the program as sophomores. Another 73 undergraduates were admitted in 1990–91. By 1995, there were 401 undergraduates in the School of Computer Science; in fall 2013, more than 600 undergraduates made up about 37 percent of student enrollment at SCS, along with more than 600 master’s degree students.
''',
''' 
New departments, new areas of study

Along the way, the Center for Machine Translation became the Language Technologies Institute, and other new departments were formed, including the Human-Computer Interaction Institute (1993), the Institute for Software Research (1999), the Machine Learning Department (2006) and the Ray and Stephanie Lane Center for Computational Biology (2009). SCS’s seven degree-granting departments draw faculty and students from a wide variety of disciplines, including engineering, mathematics, social sciences, linguistics and design.
''',
''' 
Committed to extending our founders’ vision

The School of Computer Science at Carnegie Mellon University enters its second quarter century as a world-leading educational and research institution, embracing all facets of computing. Its graduate programs are consisted ranked with the best in the world by a leading U.S. magazine, while its undergraduate programs are also rated the best in the U.S. by corporate recruiters. In 2013, SCS had 284 faculty members and a total student enrollment of nearly 1,700, including undergraduate, master’s and Ph.D. students, and conducted $124 million in research. Indeed, by itself, the Robotics Institute is the largest university robotics research group in the world, with more than 500 people and more than 100 ongoing research projects. A half-century ago, Perlis, Simon and Newell outlined a vision for computer science. The School of Computer Science at CMU remains committed to continuing and extending their vision in the context of big data and connected computing in the 21st century.
'''
]

In [9]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/scs_history.json', 'w') as f:
    json.dump(text, f)

# CMU Fact Sheet

In [35]:
#use pypdf to parse an online pdf
url = r'https://www.cmu.edu/about/cmu_fact_sheet_02.pdf'
import io
import requests
from PyPDF2 import PdfReader
headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

response = requests.get(url=url, headers=headers, timeout=120)
on_fly_mem_obj = io.BytesIO(response.content)
pdf_file = PdfReader(on_fly_mem_obj)

In [41]:
text = ''
for p in pdf_file.pages:
    text += p.extract_text()

In [43]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/fact_sheet.json', 'w') as f:
    json.dump([text], f)

# CMU History and Buggy

In [6]:
import requests
from bs4 import BeautifulSoup
url = r'https://www.cmu.edu/about/history.html'
#parse the whole webpage's text
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
text = soup.get_text()

In [9]:
text.index('Andrew Carnegie')

326

In [10]:
text.index('COVID-19')

6674

In [11]:
text = text[326:6674].split('\n')
text = [t for t in text if t != '']
text = '\n'.join(text)
text

'Andrew Carnegie \nA self-educated "working boy" who loved books, Andrew Carnegie emigrated from Scotland in 1848 and settled in Pittsburgh, Pa. Attending night school and borrowing books, Carnegie went from factory worker in a textile mill to successful entrepreneur and industrialist. He rose to prominence by founding what became the world\'s largest steel producing company by the end of the 19th century.\n \n Carnegie Technical Schools \nAt one point the richest man in the world, Carnegie believed that "to die rich is to die disgraced." He turned his attention to writing, social activism and philanthropy, determined to establish educational opportunities for the general public where few existed.\nIn 1900, he donated $1 million for the creation of a technical institute for the city of Pittsburgh, envisioning a school where working-class men and women of Pittsburgh could learn practical skills, trades and crafts that would enhance their careers, lives and communities.\n"My heart is in 

In [12]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/cmu_history.json', 'w') as f:
    json.dump([text], f)

In [20]:
url = r'https://www.cmu.edu/news/stories/archives/2019/april/spring-carnival-buggy.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
text = soup.get_text()

In [21]:
text.index('Buggy Races Keep Rolling at Carnegie Mellon'), text.index('around the course')

(6, 6938)

In [22]:
text = text[6: 6970]
text = text.split('\n')
text = [t for t in text if t != '']
text = '\n'.join(text)
text


'Buggy Races Keep Rolling at Carnegie Mellon -     News - Carnegie Mellon University\nCarnegie Mellon University\n——— \nSearch\n Search\n Search this site only \nNews\n \nNews\n                                     \xa0›\xa0             Stories\n    \n                                                 \xa0›\xa0             Archives\n    \n                                                 \xa0›\xa0             2019\n    \n                                                 \xa0›\xa0             April\n    \n                                                 \xa0›\xa0             Buggy Races Keep Rolling at Carnegie Mellon\n                            \nApril 10, 2019\nBuggy Races Keep Rolling at Carnegie Mellon\nIn its 99th year, the tradition is a Spring Carnival treat\nBy Heidi Opdyke opdyke(through)andrew.cmu.edu\nMedia Inquiries \nJulie Mattera\nMarketing and Communications\n jmattera(through)cmu.edu\n412-268-2902\n \nSweepstakes, also known as the Buggy Races, has come a long way at Carnegi

In [23]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/buggy_history.json', 'w') as f:
    json.dump([text], f)

# Tartan, Scotty, and Kiltie

In [34]:
#it's really just easier to copy the whole page 
text = '''Tartan Facts: Who founded Carnegie Mellon University? Carnegie Technical Schools was founded in 1900 by Andrew Carnegie. Twelve years later it became known as the Carnegie Institute of Technology. In 1967, the school merged with Mellon Institute and became what is known today as Carnegie Mellon University. What is a Tartan? The Carnegie Mellon athletic teams are nicknamed the "Tartans" as a nod to Andrew Carnegie's Scottish heritage. A tartan is often misrepresented as a fierce warrior from either the Asian tundra or Scottish highlands. In actuality, a Tartan is a twilled woolen fabric with a plaid design. It is of Scottish origin and consists of stripes of various colors and widths against a solid ground, denoting a particular family lineage. The school's founder, Andrew Carnegie, was born in Dunfermline, Scotland, in 1835. Carnegie came to the United States in 1848 and founded Carnegie Technical Schools in Pittsburgh in 1900.

The Scottish terrier mascot performer sports Carnegie tartan attire, while the graphic mascot is wearing a plaid scarf around its neck. So what's the difference between tartan and plaid?

You'll know it's a tartan if...• "It's a check or pattern in a variety of colours in woven fabric in which bands of colour are repeated in equal proportion in warp (running lengthwise) and weft (running across)."
• "Each stripe of the warp crosses every stripe of the weft, so when vertical and horizontal stripes of the same color cross, the result is solid color at the point of intersection."
• "The arrangement of colored threads is the same in the warp as in the weft."

You can find our official tartan on various items in the University Store.

Source: "Tartan: Romancing the Plaid," by Jeffrey Banks and Doria De La Chapelle 

 

Official Mascot?
More than a century after Carnegie Mellon University opened its doors, an official mascot finally made its mark. Although students have dressed as a Scottish terrier — typically referred to as Scotty — for 50 years, it wasn't until 2007 that Carnegie Mellon officially welcomed the Scottish terrier as the university's first mascot.

In keeping with tradition, students, faculty, staff and alumni voted to name the new official mascot Scotty. But it's not just the costumed mascot that voters named. The live dog is also known as Scotty to the Carnegie Mellon community.

About Scotty | Celebrating Scotty | Performer Requests

 

Department Quick Facts

Name of School: Carnegie Mellon University

City/Zip: Pittsburgh, PA 15213

Founded: 1900

Enrollment (undergrad): 6,982- Fall 2021

Enrollment (graduate and doctoral): 7,062 - Fall 2021

Alumni: 102,577

Nickname: Tartans

Mascot: Scottie Dog

School Colors: Cardinal and Gray

Football Stadium and Track and Field: Gesling Stadium

Capacity: 3,500

Surface: FieldTurf

Basketball/Volleyball Gym: Wiegand Gymnasium

Capacity: 500

Soccer Stadium: CMU Soccer Field

Capacity: 250 Surface: FieldTurf Affiliation: NCAA Division III Conference: University Athletic Association | Presidents' Athletic Conference (Football Only) President: Dr. Farnam Jahanian Alma Mater, Year: University of Texas at Austin Athletic Director: Dr. Josh Centor Alma Mater, Year: Brandeis, 2004 Athletic Department Phone: 412-268-8054 Mailing Address:  5000 Forbes Avenue / Pittsburgh, PA 15213
'''

In [35]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/tartan_fact.json', 'w') as f:
    json.dump([text], f)

In [36]:
text = '''About Scotty

The Scottish terrier has long been a familiar figure around Carnegie Mellon's campus. For years students have suited up in an unofficial Scottish terrier costume to excite the fans at athletic events. But the relationship between the Scottish terrier breed and Carnegie Mellon far precedes anybody doing somersaults in a dog costume. Andrew Carnegie, founder of the university, kept a Scottish terrier as his pet.

Scotty's road from popular icon to official mascot of the university began in 2006. Carnegie Mellon formed a Mascot Identity Task Force in November 2006, which consisted of students, faculty, staff and alumni. The Task Force was co-chaired by Director of Athletics Susan Bassett and Dean of Student Affairs Jennifer Church.

The mascot selection process included a series of surveys and a university Town Hall meeting. Nearly 78 percent of 2,370 students surveyed in February 2007 voted for the Scottish terrier, and approximately 25 percent of 400 alumni surveyed thought the Scottish terrier was already the mascot.

In the spring, the Task Force partnered with SME Branding — a firm with more than 17 years of experience creating mascots for professional sports teams and universities — to develop the graphics for the mascot. During October, students and alumni reviewed potential mascot images in focus groups.

Carnegie Mellon's official mascot debuted at the Nov. 10, 2007 home football game. The graphic features a profile of a distinguished, bold Scottish terrier sporting a plaid scarf around his neck. The dog is contained in a shield, representing Carnegie Mellon's Scottish heritage.

The Task Force then partnered with a mascot costume company to design our Scottish terrier in the winter of 2007. The official Scotty costume was unveiled at the 2008 Spring Carnival.

Fun Fact: The Scottish terrier breed is known for its keen, alert and intelligent expression. Its temperament is described as determined and thoughtful while its physical aspects exemplify strength, power and agility in a small package. Many of these traits are also apparent throughout the university, making the Scottish terrier a natural choice for Carnegie Mellon's mascot.
'''

In [37]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/scotty_fact.json', 'w') as f:
    json.dump([text], f)

In [38]:
text = '''The Kiltie Band

The Kiltie Band began in 1908 with a group of just seven students dedicated to supporting Carnegie Tech football, and today's Kiltie Band continues a tradition of excellence originated over a century ago. The Band took the field for its first official performance on November 25th, 1922, on what would have been Andrew Carnegie's 87th birthday.

The Kiltie Marching Band consists of musicians and colorguard and plays at all Carnegie Mellon home football games. There is a new show every week with music ranging from traditional marches and oldies to current pop tunes and jazz standards.

After football season, the Kiltie Concert Band plays a holiday concert and two spring concerts, including a performance at Carnegie Mellon's Spring Carnival. Additionally, a part of the band gets together and acts as a pep band for the basketball teams.

The Kiltie Band is a campus organization recognized by Student Senate and administered by the Department of Athletics, Physical Education and Recreation. Membership is open to all members of the campus/community without audition. Participants must do their best to promote the success of the ensemble through attendance, attitude, and dependability.
FAQs

Q: When are rehearsals?
A: Football season: Mondays and Thursdays from 5:30 p.m.-7:30 p.m. and Game Day 9:30-11:00a.m. Concert Season: Mondays and Thursdays from 5:30-6:30 p.m.
Location is the CUC Studio Theater.

Q: When are performances?
A: The Kiltie Band performs at all home football games, a holiday concert, and two spring concerts. Times and dates for all performances are announced at the first practice.

Q: Are there auditions?
A: No, any member of the campus community with music experience is able to join the Kiltie Band!

Q: Do I have to memorize music?
A: No, the music is changed for every show. You should invest in a good and trusty lyre.

Q: When is the first rehearsal?
A: On the first day of class (Monday) at 5:30 p.m. in the CUC Studio Theater. If you have questions, please email the Direcator at jolisar@andrew.cmu.edu.

Q: Can I borrow a school instrument?
A: There are several Kiltie-owned instruments available for your use. Loans begin at 4:30 p.m. before the first rehearsal. 

Q: What do they wear under those kilts?
A: Join and you'll find out!

For more information, visit the Kiltie Band website.

Interested in Joining?

Please email the following information to Kiltie Band Director Jeremy Olisar.

    Name
    High School
    Address at Carnegie Mellon (if known)
    Home address
    Cell number
    Home number
    Whether you plan on being in the band or colorguard
    If in the band what instrument(s) you play
    Whether you need to borrow an instrument or equipment
'''

In [39]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/kiltie_band_fact.json', 'w') as f:
    json.dump([text], f)

# Carnival

In [ ]:
#prepared locally using selenium

# Commencement

In [10]:
text = [
''' 
Commencement Schedule

We are proud to celebrate our newest graduates during this year’s commencement exercises!

Main Commencement Ceremony

    Bachelor’s, master’s and doctoral degree candidates and their guests are invited to join the main commencement ceremony on Sunday, May 12, for the conferral of all degrees.
    The main ceremony will include remarks from the president, keynote speaker, student speaker and academic deans, in addition to recognition of the honorary degree recipients.
    There is no limit on number of guests who can attend the main commencement ceremony and tickets are not needed.
    The ceremony will take place on CMU’s campus beginning at 10 a.m. and will be approximately 1.5 hours long. All guests must be seated by 9:15 a.m. for the start of teh student procession. Access to guest seating will be restricted once the student procession begins.
''',
''' 
Diploma Ceremonies

    In addition to the main commencement ceremony, each college/school/department will host a diploma ceremony to recognize all graduating students. Diploma ceremonies will be held over the course of the weekend (Friday, May 10–Sunday, May 12).
    Diploma ceremonies will include the presentation of diplomas to graduates, hooding of doctoral candidates and remarks from their college/school/department leadership. Each ceremony is organized and customized by their college/school/department.
    Diploma ceremonies will take place both on and off campus. Locations and times for diploma ceremonies will be provided soon.
    Diploma ceremonies typically include a reception and the length of time for each ceremony varies based on the number of graduates.

More details on the weekend schedule, including a diploma ceremony schedule, will be provided in the coming weeks.  
''',
''' 

Thursday, May 9

Phi Beta Kappa Initiation Ceremony
Ceremony: 2–3 p.m.
McConomy Auditorium, Cohon University Center
Reception: 3–4 p.m.
Connan Room, Cohon University Center

Contact:
Joseph Devine
jd0x@andrew.cmu.edu 

Joanne Ursenbach
joanneu@andrew.cmu.edu 

President's Graduates Toast (bachelor's students)
3:30–4:30 p.m.
Location TBD
Registration required. Invitation, along with registration details, will be sent in late April.

First Gen Graduation Recognition
Reception: 5-5:30 p.m.
Alumni Concert Hall, College of Fine Arts
Ceremony: 5:30-6:30p.m.
Kresge Theater, College of Fine Arts

Contact:
M. Shernell Smith
mssmith@andrew.cmu.edu 

Sam Colavecchio
scolavec@andrew.cmu.edu 
412-268-7733

''',
''' 
Friday, May 10

Diploma Ceremonies
Various times

Senior Leadership Recognition Ceremony
4–5:30 p.m.
Wiegand Gym, Cohon University Center
Undergraduate students and their guests by invitation only. This ceremony recognizes nominated seniors who have reflected upon their specific leadership contributions during their time at CMU.
''',
''' 

Saturday, May 11

Diploma Ceremonies
Various times

Center for Student Diversity and Inclusion Ceremony
Noon–2:30 p.m.
Simmons Auditorium, Tepper Building

Contact:
M. Shernell Smith
mssmith@andrew.cmu.edu 

Sam Colavecchio
scolavec@andrew.cmu.edu 

Naval ROTC Commissioning
Ceremony: 1:30-2:30 p.m.
Auditorium, Soldiers & Sailors Memorial Hall & Museum *
4141 Fifth Avenue, Pittsburgh, PA 15213

Contact:
Mike Danko
mdanko@andrew.cmu.edu 

The President’s Reception in honor of CMU’s Doctoral Candidates
4–6 p.m.
Tepper Building Atrium

''',
''' 
Sunday, May 12

Gesling Stadium opens to guests
8 a.m.

Robing and procession for graduates
9–10 a.m.
Various locations across campus

Student procession begins
9:15 a.m.
All guests in stadium must be seated. Access to guest seating will be restricted once the procession begins.

Commencement Ceremony
10–11:30 a.m.
Gesling Stadium, CMU’s campus

Diploma Ceremonies
Various times
''',
''' 
What is the difference between a diploma ceremony and the main commencement ceremony?

All graduates (bachelor’s, master’s and doctoral degree candidates) will be celebrated at the main commencement ceremony on Sunday, May 12. The ceremony will include a procession of graduates, conferral of degrees, remarks from the president, keynote speaker, student speaker and academic deans, in addition to recognition of the honorary degree recipients.

In addition to the main commencement ceremony, each college/school/department will host a diploma ceremony to recognize their graduating students. Diploma ceremonies will be held over the course of the weekend (Friday, May 10–Sunday, May 12) and include the presentation of diplomas to graduates, hooding of doctoral candidates and remarks from their college/school/department leadership.

''',
''' 
Are there any restrictions for guests?

Guests are welcome to attend commencement exercises. There is no limit to the number of guests for the main commencement ceremony and tickets are not needed. 

There is no limit to the number of guests for the President’s Reception for Doctoral Candidates and tickets are not needed. However, we ask that all doctoral graduates register the number of guests they are planning to bring to the reception so that we can prepare accordingly. 

Please note that some diploma ceremonies will limit guests based on venue capacity. 
''',
''' 
My family and friends aren’t able to travel to Pittsburgh for commencement weekend; will the events be livestreamed?

We know how important it is to share this milestone with families near and far. Sunday’s Commencement Ceremony will be livestreamed. Diploma ceremonies will be also be live-streamed to the extent possible.

''',
''' 
Are drones or unmanned aircraft systems permitted to be operated as part of commencement?
The operation of drones is prohibited prior to, during and after the commencement ceremony on May 12. For more information about the Drone Safety Program, click here. 
''',
''' 
Graduates

Congratulations on your upcoming graduation! We can't wait to celebrate with you during Commencement, May 10 – 12, 2024.

The following groups are eligible to participate in commencement exercises:

    Bachelor’s degree students who completed their degree in August 2023, December 2023 and expected May and August 2024 graduates.
    Master’s degree students who completed their degree in August 2023, December 2023 and expected May and August 2024 graduates.
    Doctoral degree students who have defended their dissertation and completed all degree requirements in August 2023, December 2023 and expected May 2024 graduates.

As you plan for commencement weekend, refer to this page for information about deadlines, cap and gown, robing, processional and more. We will update the site with new information frequently so check back often!
''',
''' 

Diplomas

Graduating students will receive diplomas at their diploma ceremonies, which colleges/schools/departments will host throughout the weekend. We will share a final schedule in the coming weeks.  

If you are unable to attend your diploma ceremony, we will mail you your diploma. You should enter your diploma delivery addresses in SIO under Academic Info > Graduation and Diploma. Because students often move shortly after graduation, we use this address instead of your permanent address.

If you are eligible to receive honors cords and medallions, you will receive them either prior to commencement or at your diploma ceremony. We will share more details in the spring.   

If you have questions regarding diploma distribution, please contact Enrollment Services at uro-diplomas@andrew.cmu.edu.

''',
''' 

Cap & Gown

If you plan to attend the main commencement ceremony or your diploma ceremony, you should plan to purchase a cap and gown.

To purchase cap, gown and in-stock diploma frames, be sure to stop by Rangos Ballroom on March 13-14 from 10 a.m.-5 p.m. Check out graduation swag, class rings and announcements, and enter to win door prizes.

If you are unable to attend, you can purchase your regalia at the University Store or order online by April 12 to have it shipped to you in time. Regalia fees can be found here.

''',

'''

Diploma Frames, Class Rings and other Graduation Merchandise

You can purchase items to help commemorate your graduation at the University Store.

Visit the University Store website and choose the Graduation tab for more information on ordering CMU personalized class rings, personalized announcements, gifts, diploma frames and more to celebrate your graduation.

''',
''' 

Career and Professional Development Center

The Career and Professional Development Center (CPDC) offers a variety of career services for students and alumni. In addition to the services described on the CPDC website, alumni can use all of the resources available to current students, including job postings in Handshake.

''',
''' 

CMU Alumni Association

Congratulations graduating students and welcome to the Carnegie Mellon University Alumni Association! Learn how to connect with other alumni and engage with the university through events and other alumni programming by joining the CMU Alumni Community.

Following your graduation, you will begin receiving information about upcoming events, available resources, campus news and more to help introduce you to the Alumni Association. To ensure that you receive these messages, please update your contact information. And as your city, job and contact information change over time, please keep your information current so that we can stay in touch!

''',

]

In [11]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/commencement_schedule.json', 'w') as f:
    json.dump(text, f)

# Handbooks

In [12]:
#use pypdf to parse an online pdf
urls = []
import io
import requests
from PyPDF2 import PdfReader
from tqdm import tqdm

urls = [r'https://lti.cs.cmu.edu/sites/default/files/PhD_Student_Handbook_2023-2024.pdf', 
         r'https://lti.cs.cmu.edu/sites/default/files/MLT%20Student%20Handbook%202023%20-%202024.pdf',
         r'https://lti.cs.cmu.edu/sites/default/files/MIIS%20Handbook_2023%20-%202024.pdf',
         r'https://lti.cs.cmu.edu/sites/default/files/MCDS%20Handbook%2023-24%20AY.pdf',
         r'https://msaii.cs.cmu.edu/sites/default/files/Handbook-MSAII-2022-2023.pdf']
texts = []
for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

    response = requests.get(url=url, headers=headers, timeout=120)
    on_fly_mem_obj = io.BytesIO(response.content)
    pdf_file = PdfReader(on_fly_mem_obj)
    page_text = []
    for p in tqdm(pdf_file.pages):
        page_text.append( p.extract_text() )
    texts.append(page_text)

100%|██████████| 53/53 [00:00<00:00, 162.72it/s]


In [15]:
#texts is a list of list, make it a list of strings, by removing inner list, keep strings as what they are
texxts = []
for text in texts:
    texxts.extend(text)

In [20]:
import json 
with open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/handbook_texts.json', 'w') as f:
    json.dump(texxts, f)

# Dataset generation

In [1]:
import random

In [88]:
def faculty_publication(data, num_samples=50, verbose_p=0.5):
    '''
    Author: Yonatan Bisk 
    Title: HomeRobot: An Open Source Software Stack for Mobile Manipulation Research 
    Publication year: 2024 
    '''

    template = {
        "Which LTI faculty published the paper [title] in [year]?":"author",
        "Who is the author of the LTI paper [title] published in [year]?":"author",
        "What is the title of the paper published by LTI faculty [author] in [year]?":"title",
        "What is the abstract of the paper [title] published by LTI faculty [author] in [year]?":"abstract",
        "Who are the coauthors of the paper [title] published by LTI faculty [author] in [year]?":"coauthor",
    }

    result = []

    for _ in range(num_samples):
        data_dict = dict()
        data_string = random.choice(data)
        author = data_string.split("Author: ")[1].split("Title")[0].strip()
        title = data_string.split("Title: ")[1].split("Publication")[0].strip()
        year = data_string.split("Publication year: ")[1].split("Coauthors")[0].strip()
        coauthors = data_string.split("Coauthors: ")[1].split("Abstract")[0].strip()
        try:
            abstract = data_string.split("Abstract: ")[1].strip()
        except:
            abstract = None

        question = random.choice(list(template.keys()))
        question_type = template[question]

        if question_type == 'author':
            answer = author
            question = question.replace('[title]', title)
            context = data_string
   
        if question_type == 'title':
            answer = title
            question = question.replace('[author]', author)
            context = data_string
        
        if question_type == 'abstract':
            answer = abstract
            if abstract is None:
                continue 
            question = question.replace('[title]', title)
            question = question.replace('[author]', author)
            context = data_string
        
        if question_type == 'coauthor':
            answer = coauthors
            question = question.replace('[title]', title)
            question = question.replace('[author]', author)
            context = data_string
        
        question = question.replace('[year]', year)
        data_dict['question'] = question
        data_dict['answer'] = answer.strip()
        data_dict['context'] = context
        result.append(data_dict)
    return result

In [113]:
import json 
data = json.load(open('/zfsauton2/home/yifuc/11711-RAG/data/cmu/faculty_publication.json'))
faculty_publication(data, 50)

[{'question': 'Which LTI faculty published the paper Joint Modelling of Spoken Language Understanding Tasks with Integrated Dialog History in 2023?',
  'answer': 'Shinji Watanabe',
  'context': 'Author: Shinji Watanabe Title: Joint Modelling of Spoken Language Understanding Tasks with Integrated Dialog History Publication year: 2023 Coauthors: Siddhant Arora, Hayato Futami, E. Tsunoo, Brian Yan, Shinji Watanabe Abstract: Most human interactions occur in the form of spoken conversations where the semantic meaning of a given utterance depends on the context. Each utterance in spoken conversation can be represented by many semantic and speaker attributes, and there has been an interest in building Spoken Language Understanding (SLU) systems for automatically predicting these attributes. Recent work has shown that incorporating dialogue history can help advance SLU performance. However, separate models are used for each SLU task, leading to an increase in inference time and computation cos

# Langchain

In [4]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

a = Document(page_content = 'sb')

splitted = RecursiveCharacterTextSplitter(chunk_size=1000)

In [11]:
a = Document(page_content ='asoijdoiajwdoiawd')

splitted.split_documents([a, a])

[Document(page_content='asoijdoiajwdoiawd'),
 Document(page_content='asoijdoiajwdoiawd')]

In [16]:
def _make_w_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f_dirname = os.path.dirname(f)
        if f_dirname != "":
            os.makedirs(f_dirname, exist_ok=True)
        f = open(f, mode=mode)
    return f


def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f

def jdump(obj, f, mode="w", indent=4, default=str):
    """Dump a str or dictionary to a file in json format.

    Args:
        obj: An object to be written.
        f: A string path to the location on disk.
        mode: Mode for opening the file.
        indent: Indent for storing json dictionaries.
        default: A function to handle non-serializable entries; defaults to `str`.
    """
    f = _make_w_io_base(f, mode)
    if isinstance(obj, (dict, list)):
        json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str):
        f.write(obj)
    else:
        raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()

def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict

In [21]:
from os import listdir
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import io 
import json 

def load_documents(text_path, chunk_size=1000, chunk_overlap=150):
    '''
    text_path: str, a directory of .txt files
    return: 
    langchain document format (a list of trucated document)
    '''
    if 'databrick' in text_path:
        loader = DirectoryLoader(text_path, glob="**/*.txt")
        data = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        docs = text_splitter.split_documents(data)
        return docs
    elif 'cmu' in text_path:
        docs = []
        #files that need to be truncated
        truncate_files = ['buggy_history.json', 'cmu_history.json', 'handbook_text.json', 'kiltie_band_fact.json',
                          'scotty_fact.json', 'tartan_fact.json']
        files = listdir(text_path)
        docs = []
        for file in files:
            doc = jload(text_path + '/' + file)
            doc = [Document(page_content = s) for s in doc]
            if file not in truncate_files:
                docs.extend(doc)
            else:
                text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
                doc = text_splitter.split_documents(doc)
                docs.extend(doc)
        return docs

In [22]:
load_documents('/zfsauton2/home/yifuc/11711-RAG/data/cmu')

[Document(page_content='Tartan Facts: Who founded Carnegie Mellon University? Carnegie Technical Schools was founded in 1900 by Andrew Carnegie. Twelve years later it became known as the Carnegie Institute of Technology. In 1967, the school merged with Mellon Institute and became what is known today as Carnegie Mellon University. What is a Tartan? The Carnegie Mellon athletic teams are nicknamed the "Tartans" as a nod to Andrew Carnegie\'s Scottish heritage. A tartan is often misrepresented as a fierce warrior from either the Asian tundra or Scottish highlands. In actuality, a Tartan is a twilled woolen fabric with a plaid design. It is of Scottish origin and consists of stripes of various colors and widths against a solid ground, denoting a particular family lineage. The school\'s founder, Andrew Carnegie, was born in Dunfermline, Scotland, in 1835. Carnegie came to the United States in 1848 and founded Carnegie Technical Schools in Pittsburgh in 1900.'),
 Document(page_content='The S

# evaluation metric

In [1]:
import evaluate

f = evaluate.load('f1')

/zfsauton2/home/yifuc/.conda/envs/llama_hw/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
prediction = ['hihihi', 'he is bad', 'haha no sir']
answer = ['shihihit', 'he is good', 'heyhey yessir']
f.compute(references=prediction, predictions=answer)

ValueError: invalid literal for int() with base 10: 'shihihit'

In [4]:
import argparse
import collections
import json
import os
import re
import string
import sys

import numpy as np


ARTICLES_REGEX = re.compile(r"\b(a|an|the)\b", re.UNICODE)


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return ARTICLES_REGEX.sub(" ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1, recall



In [6]:
gold = 'hihihi money 123 haha oij'
pred = 'hihihi moey 123 hahk aok'

compute_exact(gold, pred)

compute_f1(gold, pred)

(0.4000000000000001, 0.4)

In [7]:
def compute_metrics(prediction, truth):
    '''
    prediction: string
    truth: string
    return:
    f1 and recall between prediction and string
    '''
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens), int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0, 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    f1 = 2 * (prec * rec) / (prec + rec)
    return f1, rec

In [8]:
compute_metrics(pred, gold)

(0.4000000000000001, 0.4)

In [13]:
import os 
result_dir = '/zfsauton2/home/yifuc/11711-RAG/result/cmu'
df = {'Dataset': [], 'Encoder': [], 'VectorDB': [], 'Decoder': [], 'F1': [], 'Recall': [], 'Retrieval Accuracy': []}
for file in os.listdir(result_dir):
    if file.endswith('.json'):
        items = file.split('_')
        encoder = items[-4]
        vectordb = items[-5]
        decoder = items[-6]
        print(f'encoder: {encoder}')
        print(f'vectordb: {vectordb}')
        print(f'decoder: {decoder}')
        print('items')
        
        if 'automate' in file:
            ds_name = 'automated dataset'
        else:
            ds_name = 'manual dataset'
        
        result = json.load(open(result_dir + '/' + file))[-1]
        f1, recall, retrieval_acc = result['f1'], result['recall'], result['retrieval_acc']
        df['Dataset'].append(ds_name)
        df['Encoder'].append(encoder)
        df['VectorDB'].append(vectordb)
        df['Decoder'].append(decoder)
        df['F1'].append(f1)
        df['Recall'].append(recall)
        df['Retrieval Accuracy'].append(retrieval_acc)

import pandas as pd
df = pd.DataFrame(df)
df.to_csv('/zfsauton2/home/yifuc/11711-RAG/result/cmu.csv', index=False)

encoder: GIST-large-Embedding-v0
vectordb: FAISS
decoder: phi-2
items
encoder: e5-mistral-7b-instruct
vectordb: Chroma
decoder: phi-2
items
encoder: contriever
vectordb: FAISS
decoder: phi-2
items
encoder: e5-mistral-7b-instruct
vectordb: Chroma
decoder: phi-2
items
encoder: GIST-large-Embedding-v0
vectordb: FAISS
decoder: phi-2
items
encoder: e5-mistral-7b-instruct
vectordb: FAISS
decoder: phi-2
items
encoder: e5-mistral-7b-instruct
vectordb: FAISS
decoder: phi-2
items
encoder: contriever
vectordb: FAISS
decoder: phi-2
items
encoder: SFR-Embedding-Mistral
vectordb: Chroma
decoder: phi-2
items
encoder: GIST-large-Embedding-v0
vectordb: Chroma
decoder: phi-2
items
encoder: contriever
vectordb: Chroma
decoder: phi-2
items
encoder: SFR-Embedding-Mistral
vectordb: FAISS
decoder: phi-2
items
encoder: GIST-large-Embedding-v0
vectordb: Chroma
decoder: phi-2
items
encoder: SFR-Embedding-Mistral
vectordb: Chroma
decoder: phi-2
items
encoder: contriever
vectordb: Chroma
decoder: phi-2
items
enco

# ColBERT

In [1]:
from ragatouille import RAGTrainer
import json
import os
import io
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import random, numpy as np, torch
from os import listdir

/zfsauton2/home/yifuc/.conda/envs/llama_hw/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def _make_w_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f_dirname = os.path.dirname(f)
        if f_dirname != "":
            os.makedirs(f_dirname, exist_ok=True)
        f = open(f, mode=mode)
    return f


def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f

def jdump(obj, f, mode="w", indent=4, default=str):
    """Dump a str or dictionary to a file in json format.

    Args:
        obj: An object to be written.
        f: A string path to the location on disk.
        mode: Mode for opening the file.
        indent: Indent for storing json dictionaries.
        default: A function to handle non-serializable entries; defaults to `str`.
    """
    f = _make_w_io_base(f, mode)
    if isinstance(obj, (dict, list)):
        json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str):
        f.write(obj)
    else:
        raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()

def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict


In [3]:

def load_documents(text_path, chunk_size=1000, chunk_overlap=150):

    '''
    text_path: str, a directory of .txt files
    return: 
    langchain document format (a list of trucated document)
    '''
    if 'databrick' in text_path:
        loader = DirectoryLoader(text_path, glob="**/*.txt")
        data = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        docs = text_splitter.split_documents(data)
        return docs
    elif 'cmu' in text_path:
        docs = []
        #files that need to be truncated
        truncate_files = ['buggy_history.json', 'cmu_history.json', 'handbook_text.json', 'kiltie_band_fact.json',
                          'scotty_fact.json', 'tartan_fact.json']
        files = listdir(text_path)
        docs = []
        for file in files:
            doc = jload(text_path + '/' + file)
            doc = [Document(page_content = s) for s in doc]
            if file not in truncate_files:
                docs.extend(doc)
            else:
                text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
                doc = text_splitter.split_documents(doc)
                docs.extend(doc)
        return docs

In [4]:
documents = load_documents('/zfsauton2/home/yifuc/11711-RAG/data/cmu')

In [5]:
documents = [d.page_content for d in documents]

In [ ]:
#evaluate zero shot retrieval 
from ragatouille import RAGPretrainedModel
from ragatouille.utils import get_wikipedia_page

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0", n_gpu=1)
index_path = RAG.index(index_name="/home/scratch/yifuc/checkpoints/colbert", collection=documents)

In [7]:
#evaluate zero shot retrieval 
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_index("/home/scratch/yifuc/checkpoints/colbert")
manual_data = jload('/zfsauton2/home/yifuc/11711-RAG/data/questions.json')
automate_data = jload('/zfsauton2/home/yifuc/11711-RAG/data/automated_questions.json')

manual_correct, manual_total = 0, 0
for data in manual_data:
    manual_total += 1
    question = data['question']
    results = RAG.search(question)
    for result in results:
        if data['answer'] in result['content']:
            manual_correct += 1
            break
print(f'manual accu: {manual_correct/manual_total}')

automate_correct, automate_total = 0, 0
for data in automate_data:
    automate_total += 1
    question = data['question']
    results = RAG.search(question)
    for result in results:
        if data['answer'] in result['content']:
            automate_correct += 1
            break
print(f'automate accu: {automate_correct/automate_total}')


Loading searcher for index /home/scratch/yifuc/checkpoints/colbert for the first time... This may take a few seconds
[Feb 24, 17:46:15] #> Loading codec...
[Feb 24, 17:46:15] #> Loading IVF...
[Feb 24, 17:46:16] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 2130.17it/s]

[Feb 24, 17:46:16] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 72.55it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Where can a mural celebrating Buggy be found within CMU, and what does it depict?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2073,  2064,  1037, 15533, 12964, 11829,  6292,  2022,
         2179,  2306,  4642,  2226,  1010,  1998,  2054,  2515,  2009, 17120,
         1029,   102,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



manual accu: 0.3333333333333333
automate accu: 0.5540540540540541


In [6]:
pairs = []
manual_data = jload('/zfsauton2/home/yifuc/11711-RAG/data/questions.json')
automate_data = jload('/zfsauton2/home/yifuc/11711-RAG/data/automated_questions.json')
automate_manual_data = jload('/zfsauton2/home/yifuc/11711-RAG/data/augmented_data/generated_dataset_with_facts_5000.json')

for data in automate_manual_data:
    pairs.append((data['question'], data['context']))


In [8]:
from ragatouille import RAGTrainer
from ragatouille.utils import get_wikipedia_page


trainer = RAGTrainer(model_name = "fintunedonautomate",
        pretrained_model_name = "colbert-ir/colbertv2.0", n_usable_gpus=1) # In this example, we run fine-tuning

# This step handles all the data processing, check the examples for more details!
trainer.prepare_training_data(raw_data=pairs,
                                data_out_path="/home/scratch/yifuc/data/colbert",
                                all_documents=documents)

trainer.train(batch_size=32) # Train with the default hyperparams

Loading Hard Negative SimpleMiner dense embedding model BAAI/bge-small-en-v1.5...
Building hard negative index for 20173 documents...
All documents embedded, now adding to index...
save_index set to False, skipping saving hard negative index
Hard negative index generated
#> Starting...
nranks = 1 	 num_gpus = 8 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 32,
    "accumsteps": 1,
    "lr": 5e-6,
    "maxsteps": 500000,
    "save_every": 122,
    "warmup": 122,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name"

/zfsauton2/home/yifuc/.conda/envs/llama_hw/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#> LR will use 122 warmup steps and linear decay over 500000 steps.

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What is the proposed solution for enhancing mental health care for refugees?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2003,  1996,  3818,  5576,  2005, 20226,  5177,
         2740,  2729,  2005,  8711,  1029,   102,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

				 0.4289298355579376 0.3925057351589203
#>>>    20.81 13.01 		|		 7.799999999999999
[Feb 25, 09:01:11] 0 0.8214355707168579
				 0.28376349806785583 0.6383675932884216
#>>>    20.83 13.51 		|		 7.3199999999999985
[Feb 25, 09:01:11] 1 0.8215362662076949
				 0.093493901193141

In [9]:
#evaluate zero shot retrieval 
from ragatouille import RAGPretrainedModel
from ragatouille.utils import get_wikipedia_page

RAG = RAGPretrainedModel.from_pretrained("/zfsauton2/home/yifuc/11711-RAG/experiment/.ragatouille/colbert/none/2024-02/25/08.57.38/checkpoints/colbert/", n_gpu=1)
index_path = RAG.index(index_name="/home/scratch/yifuc/checkpoints/colbert_ft2", collection=documents)

________________________________________________________________________________
WARNING! You have a GPU available, but only `faiss-cpu` is currently installed.
 This means that indexing will be slow. To make use of your GPU.
Please install `faiss-gpu` by running:
pip uninstall --y faiss-cpu & pip install faiss-gpu
 ________________________________________________________________________________
Will continue with CPU indexing in 5 seconds...


[Feb 25, 09:10:56] #> Creating directory /home/scratch/yifuc/checkpoints/colbert_ft2 


[Feb 25, 09:10:56] [0] 		 #> Encoding 20912 passages..
[Feb 25, 09:11:10] [0] 		 avg_doclen_est = 56.90211486816406 	 len(local_sample) = 20,912
[Feb 25, 09:11:10] [0] 		 Creating 16,384 partitions.
[Feb 25, 09:11:10] [0] 		 *Estimated* 1,189,937 embeddings.
[Feb 25, 09:11:10] [0] 		 #> Saving the indexing plan to /home/scratch/yifuc/checkpoints/colbert_ft2/plan.json ..
Clustering 1139937 points in 128D to 16384 clusters, redo 1 times, 20 iterations
  Preproc

0it [00:00, ?it/s]

[Feb 25, 09:50:55] [0] 		 #> Encoding 20912 passages..


1it [00:13, 13.76s/it]
100%|██████████| 1/1 [00:00<00:00, 670.66it/s]

[Feb 25, 09:51:09] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 25, 09:51:09] #> Building the emb2pid mapping..
[Feb 25, 09:51:09] len(emb2pid) = 1189937



100%|██████████| 16384/16384 [00:00<00:00, 104372.02it/s]

[Feb 25, 09:51:09] #> Saved optimized IVF to /home/scratch/yifuc/checkpoints/colbert_ft2/ivf.pid.pt


Done indexing!


In [11]:
#evaluate zero shot retrieval 
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_index("/home/scratch/yifuc/checkpoints/colbert_ft2")
manual_data = jload('/zfsauton2/home/yifuc/11711-RAG/data/questions.json')
automate_data = jload('/zfsauton2/home/yifuc/11711-RAG/data/automated_questions.json')

manual_correct, manual_total = 0, 0
for data in manual_data:
    manual_total += 1
    question = data['question']
    results = RAG.search(question)
    for result in results:
        if data['answer'] in result['content']:
            manual_correct += 1
            break
print(f'manual accu: {manual_correct/manual_total}')

automate_correct, automate_total = 0, 0
for data in automate_data:
    automate_total += 1
    question = data['question']
    results = RAG.search(question)
    for result in results:
        if data['answer'] in result['content']:
            automate_correct += 1
            break
print(f'automate accu: {automate_correct/automate_total}')


Loading searcher for index /home/scratch/yifuc/checkpoints/colbert_ft2 for the first time... This may take a few seconds
[Feb 25, 10:04:32] #> Loading codec...
[Feb 25, 10:04:32] #> Loading IVF...
[Feb 25, 10:04:32] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1993.49it/s]

[Feb 25, 10:04:32] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 95.13it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Where can a mural celebrating Buggy be found within CMU, and what does it depict?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2073,  2064,  1037, 15533, 12964, 11829,  6292,  2022,
         2179,  2306,  4642,  2226,  1010,  1998,  2054,  2515,  2009, 17120,
         1029,   102,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



manual accu: 0.3333333333333333
automate accu: 0.6081081081081081


# reranker

In [1]:
def reranker(reranker_model, tokenizer, question, documents):
    '''
    Reranks the documents based on the question
    '''
    pairs = [[question, doc] for doc in documents]
    with torch.no_grad():
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512).to(reranker_model.device)
        scores = reranker_model(**inputs, return_dict=True).logits.view(-1, ).float()
        return scores


In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    HfArgumentParser,
    GenerationConfig,
    AutoModelForSequenceClassification
)
reranking_prompt = "<s>[INST]Please rank the following documents based on their relevance to the question. Output rank in a python list format. Below is the question and the documents.\n Question: {question}\nDocuments: {context}\n Output: [/INST]"
reranker_tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral', padding_side='left', trust_remote_code=True)
reranker_model = AutoModelForSequenceClassification.from_pretrained('Salesforce/SFR-Embedding-Mistral', 
                                                                    trust_remote_code=True).to('cuda:7')
reranker_model.eval()

passage = ['apple is bad', 'apple is good', 'banana is good', 'cat loves apple', 'banana is good for health', 'apple is bad for health']
question = 'is apple good for health?'

documents = [f'Document {i+1}. {p}' for i, p in enumerate(passage)]
documents = ' '.join(documents)

p = reranking_prompt.format(question=question, context=documents)

generation_config = GenerationConfig(
    max_length=4096, temperature=0.01, top_p=0.95, repetition_penalty=1.1,
    do_sample=True, use_cache=True,
    eos_token_id=reranker_tokenizer.eos_token_id, pad_token_id=reranker_tokenizer.pad_token_id,
)

inputs_with_doc = reranker_tokenizer(p, return_tensors="pt", return_attention_mask=False).to(reranker_model.device)
answers = reranker_model.generate(**inputs_with_doc, pad_token_id=reranker_tokenizer.eos_token_id, generation_config=generation_config)
answers = reranker_tokenizer.batch_decode(answers)

/zfsauton2/home/yifuc/.conda/envs/llama_hw/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.41s/it]
Some weights of MistralForCausalLM were not initialized from the model checkpoint at Salesforce/SFR-Embedding-Mistral and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    HfArgumentParser,
    GenerationConfig,
    AutoModelForSequenceClassification
)
import torch
reranking_prompt = "<s>[INST]Please rank the following documents based on their relevance to the question. Output rank in a python list format. Below is the question and the documents.\n Question: {question}\nDocuments: {context}\n Output: [/INST]"
reranker_tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral', padding_side='left', trust_remote_code=True)
reranker_model = AutoModelForSequenceClassification.from_pretrained('Salesforce/SFR-Embedding-Mistral', 
                                                                    trust_remote_code=True).to('cuda:7')
reranker_model.eval()

passage = ['apple is bad', 'apple is good', 'banana is good', 'cat loves apple', 'banana is good for health', 'apple is bad for health']
question = 'is apple good for health?'

reranker(reranker_model, reranker_tokenizer, question, passage)


/zfsauton2/home/yifuc/.conda/envs/llama_hw/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.67s/it]
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at Salesforce/SFR-Embedding-Mistral and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([ 10.0138,  -4.5976,   8.0047,  -4.0530,   6.5506,  -1.3838,   8.7122,
         -3.0544,  -4.3724, -11.6452,   9.6951,  -5.4365], device='cuda:7')